<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/RotinasDeLimpezaDaBaseDeDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Base de Dados Twitter

Análise da base de dados do Twitter com linguagem ofensiva

Pré - Processamento do texto

Classificação

Import e instalação das bibliotecas necessárias

In [ ]:
#Install e downloads
!pip install emoji
!pip install pandas
!pip install nltk
#nltk.download('all-nltk') #Demora um pouco

     |████████████████████████████████| 174 kB 5.1 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=3318b57d50ba90e1ba53b541b4c942ac10afbca3eaa4dd018c55921c84aeed68
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [2]:
#Imports
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import re
import itertools

In [94]:
# Definição de constantes

# Padrão RegEx
PADRAO_PALAVRAS_PT = r'[a-zA-Zà-úÀ-Ú0-9]+'

# Quantidade tokens tweets pequeno
QUANT_TOKENS_MIN = 5

########## PATHS DE ARQUIVOS ##########
# Base completa bruta
PATH_BASE_COMPLETA_BRUTA = r"/content/drive/MyDrive/TCC/dados/tweets_coletados/base80k.csv"

# Base teste bruta
PATH_BASE_TESTE_BRUTA = r"/content/drive/MyDrive/TCC/dados/tweets_coletados/base_teste.csv"

# Dicionário de palavrões
PATH_DICT_PALAVROES = r"/content/drive/MyDrive/TCC/dados/dicionarios/badword_list.csv"

# Dicionário de internetes
PATH_DICT_INTERNETES = r"/content/drive/MyDrive/TCC/dados/dicionarios/internetes_list.csv"

# Dicionário de estados
PATH_DICT_ESTADOS = r"/content/drive/MyDrive/TCC/dados/dicionarios/estados_list.csv"

# Dicionário de expressões contraídas
PATH_DICT_EXPRESSOES = r"/content/drive/MyDrive/TCC/dados/dicionarios/dict_hate_contraidos.csv"

# Lista de vogais duplicadas
PATH_LIST_VOGAIS_DUPLICADAS = r"/content/drive/MyDrive/TCC/dados/dicionarios/vogais_duplicadas.csv"

# Lista de stopWords
PATH_LIST_STOP_WORDS_PT_BR = r"/content/drive/MyDrive/TCC/dados/dicionarios/stop_word_list.txt"



In [95]:
#Carrega a base de dados
baseTweets = pd.read_csv(PATH_BASE_COMPLETA_BRUTA, index_col=0)
#baseTweets = baseTweets.sample(n=1000) #PEGANDO 1000 REGISTROS DA BASE PARA TESTE, REMOVER ISSO DEPOIS
baseTweets.head()

,id_str,created_at,text,entities.hashtags,place.full_name
0,1270007273088339969,Mon Jun 08 14:58:24 +0000 2020,@allantercalivre PARABÉNS @allantercalivre,[],"Sao Paulo, Brazil"
1,1270005650945343489,Mon Jun 08 14:51:57 +0000 2020,RT @franciscoedi11: @allantercalivre Eu queria...,[],NaN
2,1270002784742539264,Mon Jun 08 14:40:34 +0000 2020,@allantercalivre @EdnaBraga Gostei da pulseira...,[],"Sao Paulo, Brazil"
3,1270001502124417025,Mon Jun 08 14:35:28 +0000 2020,@allantercalivre @FBI Esses não irão ao encont...,[],"Recife, Brazil"
4,1270000103416266754,Mon Jun 08 14:29:54 +0000 2020,@allantercalivre Cadê eles ?\r\nTá faltando,[],"Nova Iguaçu, Brasil"


In [96]:
baseTweets.shape

(82149, 5)

Fase 1
Definição das funções usadas na limpeza Etapas:

1. Remover menções (Remover @fulano)
2. Remover caracteres especiais Remover caracteres de pontuação
3. Remover emojis
4. Expandir internetês
5. Normalizar nomes de estados (SP->São Paulo)
6. Retirar letras repetidas
7. Remover tweets com <5 tokens
8. Remover tweets repetidos (texto idêntico)

In [97]:
# Main

# Carregando dicionários
tokenizer = RegexpTokenizer(PADRAO_PALAVRAS_PT)
dict_internetes = pd.read_csv(PATH_DICT_INTERNETES)
dict_estados = pd.read_csv(PATH_DICT_ESTADOS)
dict_expressoes = pd.read_csv(PATH_DICT_EXPRESSOES, encoding='latin-1')
list_vogais_duplicadas = pd.read_csv(PATH_LIST_VOGAIS_DUPLICADAS)

def pre_processamento(text):
    # Limpeza
    text = remover_mencao(text.lower()) # Remover menções
    text = re.sub(r'http\S+', '', text) # Remover links
    tokens = tokenizer.tokenize(text) # Remover caracteres especiais, pontuação e emojis
    tokens = expandir_sigla(tokens,dict_internetes) # Expandir Internetes
    tokens = expandir_sigla(tokens,dict_estados) # Normalizar nomes de estado
    tokens = remover_letras_repetidas(tokens, list_vogais_duplicadas) # Retirar letras repetidas
    tokens = seta_tweet_pequeno_para_um(tokens) # Marca tweets pequenos

    text_limpo = " ".join(tokens)
    return text_limpo

# Remover menções   
def remover_mencao(tweet):
    tweet_sem_mencao = re.sub(r'[@]\w+', '', tweet)
    tweet_sem_mencao = " ".join(tweet_sem_mencao.split()) # Remover espaços a mais
    return tweet_sem_mencao

# ESSA FUNÇÃO É CASE-SENSITIVE
def expandir_sigla(tokens, dicionario):
    colunas_dicionario = dicionario.columns
    dicionario_py = dicionario.set_index(colunas_dicionario[0]).to_dict() # Converte Dataframe para dicionário python
    dicionario_py = dicionario_py[colunas_dicionario[1]]
    keys = list(dicionario_py.keys())

    indice = 0
    for token in tokens:
      if token in keys:
        tokens[indice] = dicionario_py[token]
      indice+=1
    
    return tokens

# Retirar letras repetidas
def remover_letras_repetidas(tokens, lista):
    lista = list(lista['vogais_duplicadas'])
    tokens_sem_letras_repetidas = []
    for token in tokens:
        flag = True
        if token not in lista: # Se for uma palavra com vogais dobradas não modifica o token
            for i in range(len(token)):
                # Se tiver "rr" ou "ss" não modifica o token
                try:
                    if (token[i].lower()=='r' and token[i+1].lower()=='r') or\
                        (token[i].lower()=='s' and token[i+1].lower()=='s'):
                        flag = False
                        break
                except:
                    break
            if flag:
                # Remove letras repetidas
                token = ''.join(c[0] for c in itertools.groupby(token))
        tokens_sem_letras_repetidas.append(token)
    return tokens_sem_letras_repetidas

# Marca tweets pequenos
def seta_tweet_pequeno_para_um(tokens):
    if len(tokens)<QUANT_TOKENS_MIN:
        tokens = '1'
    return tokens

In [98]:
# Aplicar pré processamento na base
baseTweets['text_limpo'] = baseTweets['text'].apply(lambda x: pre_processamento(x))
baseTweets.head()

In [99]:
# Remover tweets marcados como pequenos e remove tweets repetidos(texto idêntico) 
baseTweets.drop_duplicates(subset='text_limpo', keep='first', inplace=True)

# apaga a linha com o texto = '1' que sobra
baseTweets = baseTweets[baseTweets['text_limpo']!='1']

# consertar o index
#baseTweets.reset_index(inplace=True)
#baseTweets.drop(columns="index" , inplace=True)

In [100]:
# Porque está diminuindo quando rodamos mais de uma vez ?
baseTweets.shape

(63380, 6)

In [102]:
baseTweets.to_csv('/content/drive/MyDrive/TCC/dados/processadas/Base1_Com_StopWords.csv') #Salva base limpa

Fase 2
remover stopswords

In [103]:
# O arquivo "portuguese" contem Stopwords em Portugues
Sw = open(PATH_LIST_STOP_WORDS_PT_BR,'r',encoding='utf-8')
ptBr_stopwords = Sw.read()

In [104]:
baseTweets['text_cleanStopWords'] = baseTweets['text_limpo'].apply(lambda x: ' '.join([word for word in x.split() if word not in (ptBr_stopwords)]))
baseTweets.head()

,id_str,created_at,text,entities.hashtags,place.full_name,text_limpo,text_cleanStopWords
1,1270005650945343489,Mon Jun 08 14:51:57 +0000 2020,RT @franciscoedi11: @allantercalivre Eu queria...,[],NaN,retwet eu queria compartilhar mas antes eu gos...,retwet queria compartilhar antes gostaria sabe...
2,1270002784742539264,Mon Jun 08 14:40:34 +0000 2020,@allantercalivre @EdnaBraga Gostei da pulseira...,[],"Sao Paulo, Brazil",gostei da pulseira e da educação dele nossaaa ...,gostei pulseira educação nossaaa muita elegânc...
3,1270001502124417025,Mon Jun 08 14:35:28 +0000 2020,@allantercalivre @FBI Esses não irão ao encont...,[],"Recife, Brazil",esses não irão ao encontro e beijing,irão encontro beijing
5,1269998146601529344,Mon Jun 08 14:22:08 +0000 2020,@allantercalivre e eu me questionando: “quem ...,[],"Mossoró, Brasil",e eu me questionando quem é esse blogueiro sem...,questionando blogueiro blog juntos
6,1269997249746087939,Mon Jun 08 14:18:34 +0000 2020,@allantercalivre @marciolabre @jairbolsonaro R...,[],"Uberlândia, Brasil",renan caloteiro desonesto vejam link abaixo,renan caloteiro desonesto vejam link abaixo


In [105]:
baseTweets.to_csv('/content/drive/MyDrive/TCC/dados/processadas/Base2_Sem_StopWords.csv') #Salva base limpa